In [1]:
!pip install -qU datasets pinecone-client sentence-transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43

In [2]:
from datasets import load_dataset

wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

In [3]:
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [4]:
history = wiki_data.filter(
    lambda d: d['section_title'].startswith('History')
)

In [5]:
from tqdm.auto import tqdm

total_doc_count = 50000

counter = 0
docs = []
for d in tqdm(history, total=total_doc_count):
  doc = {
      "article_title" : d["article_title"],
      "section_title" : d["section_title"],
      "passage_text"  : d["passage_text"]
  }
  docs.append(doc)
  if counter == total_doc_count:
    break
  counter+=1

  0%|          | 0/50000 [00:00<?, ?it/s]

In [6]:
import pandas as pd

df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,Taupo District,History,was not until the 1950s that the region starte...
1,Sutarfeni,History & Western asian analogues,Sutarfeni History strand-like pheni were Phena...
2,The Bishop Wand Church of England School,History,The Bishop Wand Church of England School Histo...
3,Teufelsmoor,History & Situation today,"made to preserve the original landscape, altho..."
4,Surface Hill Uniting Church,History,in perpetual reminder that work and worship go...


In [7]:
from sentence_transformers import SentenceTransformer

retrever = SentenceTransformer(
    "flax-sentence-embeddings/all_datasets_v3_mpnet-base"
)

In [8]:
retrever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [9]:
api_key = "api_key"

In [10]:
import pinecone

pinecone.init(
    api_key = api_key,
    environment = "gcp-starter"
)

In [12]:
index_name = "question-answer-app"

if index_name not in pinecone.list_indexes():
  pinecone.create_index(
      index_name,
      dimension = 768,
      metric = "cosine"
  )
index = pinecone.Index(index_name)

In [13]:
batch_size = 64

for i in tqdm(range(0,len(df),batch_size)):

  i_end = min(i+batch_size,len(df))
  batch = df.iloc[i:i_end]
  emb = retrever.encode(batch["passage_text"].tolist()).tolist()
  meta = batch.to_dict(orient='records')
  ids = [f"{idx}" for idx in range(i,i_end)]
  to_upsert = list(zip(ids,emb,meta))
  _ = index.upsert(vectors = to_upsert)

index.describe_index_stats()

  0%|          | 0/782 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.4992,
 'namespaces': {'': {'vector_count': 49920}},
 'total_vector_count': 49920}

In [14]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa')

In [20]:
def query_pinecone(query,top_k):
  q = retrever.encode([query]).tolist()
  context = index.query(q,top_k=top_k,include_metadata=True)
  return context

In [21]:
def format_query(query,context):
  context = [f"<P> {m[ 'metadata'][ 'passage_text']}" for m in context]
  context = " ".join(context)
  query = f"question: {query} context: {context}"
  return query

In [22]:
query = "When was the first electric power system built?"
result = query_pinecone(query,top_k=1)
result


{'matches': [{'id': '3593',
              'metadata': {'article_title': 'Electric power system',
                           'passage_text': 'Electric power system History In '
                                           '1881, two electricians built the '
                                           "world's first power system at "
                                           'Godalming in England. It was '
                                           'powered by two waterwheels and '
                                           'produced an alternating current '
                                           'that in turn supplied seven '
                                           'Siemens arc lamps at 250 volts and '
                                           '34 incandescent lamps at 40 volts. '
                                           'However, supply to the lamps was '
                                           'intermittent and in 1882 Thomas '
                                           'Ed

In [23]:
from pprint import pprint

In [24]:
query = format_query(query,result["matches"])
pprint(query)

('question: When was the first electric power system built? context: <P> '
 "Electric power system History In 1881, two electricians built the world's "
 'first power system at Godalming in England. It was powered by two '
 'waterwheels and produced an alternating current that in turn supplied seven '
 'Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. '
 'However, supply to the lamps was intermittent and in 1882 Thomas Edison and '
 'his company, The Edison Electric Light Company, developed the first '
 'steam-powered electric power station on Pearl Street in New York City. The '
 'Pearl Street Station initially powered around 3,000 lamps for 59 customers. '
 'The power station generated direct current and')


In [29]:
def generate_answers(query):
  inputs = tokenizer([query],max_length=1024,return_tensors="pt")
  ids = generator.generate(inputs["input_ids"],num_beams=2,min_length=20,max_length=40)
  answer = tokenizer.batch_decode(ids,skip_special_tokens=True,clean_up_tokenization_spaces=False)[0]
  return pprint(answer)

In [30]:
generate_answers(query)

('The first electric power system was built in 1881 at Godalming in England. '
 'It was powered by two waterwheels and produced alternating current that in '
 'turn supplied seven Siemens arc lamps')


In [33]:
query = "When was the first world war occurd"
q_result = query_pinecone(query,top_k=5)
context = format_query(query,q_result['matches'])
answer = generate_answers(context)

('The First World War was a conflict between Germany and Austria-Hungary. The '
 'war started in 1914, when Germany invaded Austria-Hungary. The war lasted '
 'until 1918, when Germany surrendered')


In [34]:
query = "When was taj mahal built"
q_result = query_pinecone(query,top_k=5)
context = format_query(query,q_result['matches'])
answer = generate_answers(context)

('The Taj Mahal was built in the 16th century. The first two towers were built '
 'in the 16th century. The third tower was built in the 17th century. The '
 'fourth tower')


In [40]:
for res in q_result["matches"]:
  print(res["metadata"]["passage_text"],end="\n\n")

this building are a precursor to the famous Taj Mahal, as the similarities can be seen. Architecture The main building displays a typical Timurid architectural style. However, today much of the building itself has suffered heavy from neglect and damage by warring factions during the Soviet and Afghan civil wars. The main courtyard has been well preserved allowing people to pray in the open spaces. To the back of the building however little care has been given to the paving and gardens. The minaret has either collapsed due to the age of the building or could have been the act

on 1 January 1996, the spires of Tower 1 and Tower 2 were completed on 1 March 1996, 3 years after its construction was started, and the first batch of Petronas personnel moved into the building on 1 January 1997. The building was officially opened by the Prime Minister of Malaysia's Tun Dr. Mahathir bin Mohamad on 1 August 1999. The twin towers were built on the site of Kuala Lumpur's race track. Test boreholes f